In [225]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/goodbooks-10k/book_tags.csv
/kaggle/input/goodbooks-10k/ratings.csv
/kaggle/input/goodbooks-10k/books.csv
/kaggle/input/goodbooks-10k/sample_book.xml
/kaggle/input/goodbooks-10k/to_read.csv
/kaggle/input/goodbooks-10k/tags.csv


In [226]:
#! pip install -Uqq fastai

# Recomendação de Livros

O objetivo artigo é criar um modelo com Filtros Colaborativos. Dado este cenário, queremos entender o comportamento de usuários e através disso perceber quais seriam as melhores recomendações. Sendo assimp ara esse caso específico desejamos fazer recomendações de livros.

In [227]:
import pandas as pd
import numpy as np
from fastai.collab import *
from fastai.tabular.all import *

## Dados

In [228]:
books = pd.read_csv('/kaggle/input/goodbooks-10k/books.csv')
ratings = pd.read_csv('/kaggle/input/goodbooks-10k/ratings.csv')

In [229]:
books.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m/2767052.jpg,https://images.gr-assets.com/books/1447303603s/2767052.jpg
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m/3.jpg,https://images.gr-assets.com/books/1474154022s/3.jpg
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m/41865.jpg,https://images.gr-assets.com/books/1361039443s/41865.jpg
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m/2657.jpg,https://images.gr-assets.com/books/1361975680s/2657.jpg
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m/4671.jpg,https://images.gr-assets.com/books/1490528560s/4671.jpg


In [230]:
print('Number of book data:', len(books.book_id.unique()))
print('Amount of Users:', len(ratings['user_id'].unique()))

Number of book data: 10000
Amount of Users: 53424


In [231]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [232]:
books['book'] = books['original_title']
drop_columns = [
    'id','best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'title', 
    'language_code', 'average_rating', 'ratings_count', 'work_ratings_count',
    'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4',
    'ratings_5', 'image_url', 'small_image_url', 'original_title', 'authors',
    'original_publication_year'
]
books.drop(columns=drop_columns, inplace=True)
books.head()

,book_id,book
0,2767052,The Hunger Games
1,3,Harry Potter and the Philosopher's Stone
2,41865,Twilight
3,2657,To Kill a Mockingbird
4,4671,The Great Gatsby


In [233]:
ratings = ratings.merge(books)
ratings.head()

,book_id,user_id,rating,book
0,1,314,5,Harry Potter and the Half-Blood Prince
1,1,439,3,Harry Potter and the Half-Blood Prince
2,1,588,5,Harry Potter and the Half-Blood Prince
3,1,1169,4,Harry Potter and the Half-Blood Prince
4,1,1185,4,Harry Potter and the Half-Blood Prince


In [266]:
pivot_table = pd.pivot_table(ratings[:1000], columns='book_id', values='rating', index='user')
pivot_table

book_id,1,2,3,5,6,8,10,11,13,21
user,,,,,,,,,,
173,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
314,5.0,NaN,3.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN
439,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
588,5.0,NaN,1.0,NaN,NaN,3.0,NaN,4.0,NaN,4.0
725,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
52583,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0
52740,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
53145,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [243]:
ratings.rename(columns={'user_id': 'user'}, inplace=True)
ratings.head()

,book_id,user,rating,book
0,1,314,5,Harry Potter and the Half-Blood Prince
1,1,439,3,Harry Potter and the Half-Blood Prince
2,1,588,5,Harry Potter and the Half-Blood Prince
3,1,1169,4,Harry Potter and the Half-Blood Prince
4,1,1185,4,Harry Potter and the Half-Blood Prince


In [310]:
ratings.describe(include=object)

,book
count,77920
unique,794
top,Harry Potter and the Half-Blood Prince
freq,100


## Modelo

In [236]:
dls = CollabDataLoaders.from_df(ratings, user_name='user',item_name='book', bs=64)
dls.show_batch()

,user,book,rating
0,31655,A Man Without a Country,3
1,9768,A Walk to Remember,3
2,53082,Galápagos,5
3,36042,The Great Train Robbery,3
4,40571,The List,3
5,18949,The Great Book of Amber,2
6,43807,Lipstick Jungle,5
7,50068,The Tragedy of Macbeth,5
8,5115,The Lord of the Rings: Weapons and Warfare,3
9,9566,The Poisonwood Bible,4


In [237]:
n_users  = len(dls.classes['user'])
n_books = len(dls.classes['book'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
book_factors = torch.randn(n_books, n_factors)

print(f'Number of users: {n_users}')
print(f'Number of books: {n_books}')
print(f'Number of factors: {n_factors}')

user_factors = torch.randn(n_users, n_factors)
book_factors = torch.randn(n_books, n_factors)

print(f'User factors shape: {user_factors.shape}')
print(f'Book factors shape: {book_factors.shape}')

Number of users: 26051
Number of books: 795
Number of factors: 5
User factors shape: torch.Size([26051, 5])
Book factors shape: torch.Size([795, 5])


In [238]:
class DotProduct(Module):
    def __init__(self, n_users, n_books, n_factors,y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.book_factors = Embedding(n_books, n_factors)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        books = self.book_factors(x[:,1])
        return sigmoid_range((users * books).sum(dim=1), *self.y_range)

In [239]:
model = DotProduct(n_users, n_books, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [245]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.100333,1.371666,00:06
1,0.142345,1.391916,00:06
2,0.217163,1.349077,00:06
3,0.057491,1.345468,00:06
4,0.021631,1.344366,00:06


In [246]:
class DotProductBias(Module):
    def __init__(self, n_users, n_books, n_factors, y_range=(0,5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1)
        self.book_factors = Embedding(n_books, n_factors)
        self.book_bias = Embedding(n_books, 1)
        self.y_range = y_range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        books = self.book_factors(x[:,1])
        res = (users * books).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:,0]) + self.book_bias(x[:,1])
        return sigmoid_range(res, *self.y_range)

In [247]:
model = DotProductBias(n_users, n_books, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.281921,1.227381,00:07
1,0.717195,0.861501,00:07
2,0.251933,0.871063,00:07
3,0.103763,0.866615,00:07
4,0.038340,0.868078,00:07


## Decadência de Peso

O objetivo é aplicar uma penalidade a pesos excessivamente altos para evitar o sobreajuste. Com base nessa informação, o modelo será reajustado durante o treinamento.

In [253]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [254]:
class DotProductBias(Module):
    def __init__(self, n_users, n_books, n_factors, y_range=(0,5.5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users])
        self.book_factors = create_params([n_books, n_factors])
        self.book_bias = create_params([n_books])
        self.y_range = y_range

    def forward(self, x):
        users = self.user_factors[x[:,0]]
        books = self.book_factors[x[:,1]]
        res = (users*books).sum(dim=1)
        res += self.user_bias[x[:,0]] + self.book_bias[x[:,1]]
        return sigmoid_range(res, *self.y_range)

In [280]:
model = DotProductBias(n_users, n_books, 50)
learn = Learner(dls, model, loss_func=MSELossFlat(), metrics=mse)
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,mse,time
0,1.338107,1.284263,1.284263,00:07
1,0.763667,0.898529,0.898529,00:07
2,0.388816,0.902006,0.902006,00:07
3,0.206338,0.888929,0.888929,00:07
4,0.097422,0.891965,0.891965,00:07


## Aprendizado Profundo

In [306]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.item_factors = Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1]+item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.item_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1))
        return sigmoid_range(x, *self.y_range)
    


embs = get_emb_sz(dls)
model = CollabNN(*embs)
    
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)

epoch,train_loss,valid_loss,time
0,0.861055,0.860495,00:14
1,0.775191,0.859854,00:11
2,0.581492,0.869260,00:11
3,0.452001,0.879909,00:11
4,0.333467,0.915948,00:11


Não houve uma melhora significativa...

In [286]:
model = DotProductBias(n_users, n_books, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(3, 5e-3, wd=10e-7)

epoch,train_loss,valid_loss,time
0,0.031630,0.890286,00:07
1,0.087318,0.872111,00:07
2,0.043484,0.867611,00:07


## Interpretando os dados

In [287]:
book_bias = learn.model.book_bias.squeeze()
idxs = book_bias.argsort()[:5]
[dls.classes['book'][i] for i in idxs]

["Harry Potter and the Philosopher's Stone",
 ' The Fellowship of the Ring',
 'Nine Stories',
 'The Pearls of Lutra',
 'The Chamber']

In [288]:
idxs = book_bias.argsort(descending=True)[:5]
[dls.classes['book'][i] for i in idxs]

['Still Life with Woodpecker',
 'Villa Incognito',
 'Girl with a Pearl Earring',
 'Tears of the Giraffe',
 'The Beautiful and Damned']

In [297]:
def get_user_preds(user_id, learn):
    rows = []
    book_infos = list(books.drop_duplicates(subset='book', keep='first').book)
    for i in range(len(book_infos)):
        rows.append(dict({'user': user_id, 'book': book_infos[i]}))
    data = pd.DataFrame(rows)
    dl = learn.dls.test_dl(data)
    preds,y = learn.get_preds(dl=dl)
    preds_df = pd.DataFrame({
        'book': book_infos,
        'prediction': preds.numpy().flatten()
        }).sort_values('prediction', ascending=False)

    return preds_df

In [298]:
preds = get_user_preds(314, learn=learn)

In [299]:
preds

,book,prediction
26,Harry Potter and the Half-Blood Prince,5.094801
23,Harry Potter and the Goblet of Fire,5.062691
822,Xenocide,4.669457
3617,"Harry Potter Collection (Harry Potter, #1-6)",4.643742
5996,Falling Angels,4.518298
...,...,...
5100,Ilium,2.004204
291,The Very Hungry Caterpillar,1.921684
3195,Quicksilver,1.876809
1042,The Woman in White,1.831199


In [300]:
hp_preds = preds.loc[preds['book'] == 'Harry Potter and the Half-Blood Prince']
hp_preds

,book,prediction
26,Harry Potter and the Half-Blood Prince,5.094801


In [301]:
hp_ratings = ratings.loc[(ratings['book'] == 'Harry Potter and the Half-Blood Prince') & (ratings['user'] == 314)]
hp_ratings

,book_id,user,rating,book
0,1,314,5,Harry Potter and the Half-Blood Prince


## Exportando

In [302]:
#exportar modelo
learn.export('model.pkl')
#exportar books
books.to_csv('books.csv', index=False)

In [305]:
#exportar users ID's
user_id = pd.DataFrame({'user_id': ratings['user'].copy()})
user_id = user_id.drop_duplicates()
user_id.to_csv('users.csv', index=False)

## Aplicação no Hugginface

https://huggingface.co/spaces/fastaioncampus/Book-Recommendation